## _Computer Vision_
### Lab 7, _Segmentation and Background Subtraction_.
#### >> Topics  :
* ##### Image Segmentation
  * ###### Adaptive Thresholding
  * ###### Otsu Thresholding
  * ###### K-Means Clustering
* ##### Background Subtraction
* ##### Motion Detection

##### >> Notes  :
> ###### To close image windows smoothly please **press Esc** on your keyboard, **don't close** it directly by clicking on 'X' to avoid kernel interruption.
> ###### To run this lab using google colab follow below instructions:
> 1. Import `cv2_imshow` method using `from google.colab.patches import cv2_imshow`.
> 2. Replace all `cv.imshow()` with `cv2_imshow()` with one parameter **name of that image**.
> 3. Specify your path.

In [61]:
import cv2 as cv
import numpy as np

### Section 0, _Helper Functions_:
> ##### This cell contains some helper function such as error_handler, please run it without any modification.

In [3]:
# please don't modify this code.
def show_text_window(titles):
    black = np.zeros((len(titles) * 150, 400))
    for idx, t in enumerate(titles):
        place = idx + 1
        cv.putText(black, t, (10, place * 100),
                   cv.FONT_HERSHEY_SIMPLEX, 0.75, (200, 0, 200), 1, 2)
    cv.imshow("Values", black)


def get_updated_value(key, value, **kwargs):
    if key == ord('+'):
        return value, 0, 0
    elif key == ord('-'):
        return -value, 0, 0
    elif key == ord('*'):
        return 0, kwargs.get('value2', value), 0
    elif key == ord('/'):
        return 0, -kwargs.get('value2', value), 0
    elif key == ord('8'):
        return 0, 0, kwargs.get('value3', value)
    elif key == ord('2'):
        return 0, 0, -kwargs.get('value3', value)
    elif key == 27:
        raise Exception('Terminated by user!')
    else:
        return 0, 0, 0


def error_handler(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as ex:
            cv.destroyAllWindows()
            print(f'{ex}')

    return wrapper

### Section 1, _Image Segmentation_:
* ##### Is the process if dividing an image into meaningful and semantically homogeneous regions or segments.
* #####  The goal is to simplify the representation of an image, making it easier to analyze and understand.
* ##### Each segment or region typically corresponds to a specific object or a part of an object within the image.
* ##### [Click to check the documentation](https://viso.ai/deep-learning/image-segmentation-using-deep-learning/)
* ![image info](https://viso.ai/wp-content/uploads/2022/06/semantic-image-segmentation-of-aerial-images-using-pytorch.jpg)

#### 1.1 Adaptive Thresholding
* ##### Is a technique where the threshold value is not fixed for the entire image but varies based on the local characteristics of the image.
* ##### This method is useful when lighting changes across different parts of the image.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html).
* ![image info](https://docs.opencv.org/3.4/ada_threshold.jpg)


In [8]:
@error_handler
def adaptive_thresholding(img_path):

    img = cv.imread(img_path, 0)
    img = cv.medianBlur(img, 5)
    ret, th1 = cv.threshold(img, 127, 255, cv.THRESH_BINARY)
    c = 10
    block_size = 11
    while True:
        th2 = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, block_size, c)
        th3 = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, c)
        titles = ['Original Image', 'Global Thresholding (v = 127)', 'AdaptiveMeanThresholding',
                  'Adaptive Gaussian Thresholding']
        images = [img, th1, th2, th3]
        for image in zip(titles, images):
            img_to_show = cv.resize(image[1], (400, 400))
            cv.imshow(image[0],img_to_show)
            cv.resizeWindow(image[0], 400, 400)
            cv.setWindowProperty(image[0], cv.WND_PROP_TOPMOST, 1)


        k = cv.waitKey(0)
        uv1, uv2, _ = get_updated_value(k, value=2, value2=2)
        block_size += uv1
        c += uv2
        block_size = max(block_size, 11)

path = "../Images/sudoku.jpg"
adaptive_thresholding(path)

Terminated by user!


#### 1.2 Otso Thresholding
* ##### It is designed to find an optimal threshold to separate an image into two classes, typically foreground and background.
* ##### The goal is to minimize the intra-class variance of pixel intensities, effectively maximizing the inter-class variance between the two classes.
* ##### This method is useful when there is a bimodal distribution of pixel intensities in the image, meaning there are clear separations between the foreground and background intensities.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html)
* * ![image info](https://scipy-lectures.org/_images/sphx_glr_plot_threshold_001.png)



In [60]:
def otsu_thresholding(path):
    img = cv.imread(path, 0)
    # global thresholding
    ret1, th1 = cv.threshold(img, 127, 255, cv.THRESH_BINARY)
    # Otsu's thresholding
    ret2, th2 = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    # Otsu's thresholding after Gaussian filtering
    blur = cv.GaussianBlur(img, (5, 5), 0)
    ret3, th3 = cv.threshold(blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    images = [img, th1, th2, th3]
    titles = ['Original Image', 'Binary Threshold', 'Basic OTSU Threshold', 'Gaussian OTSU Threshold']
    for image in zip(titles, images):
        cv.imshow(image[0], image[1])
        cv.setWindowProperty(image[0], cv.WND_PROP_TOPMOST, 1)
    cv.waitKey(0)
    cv.destroyAllWindows()

path = "../Images/coins_1.jpg"
otsu_thresholding(path)

#### 1.3 K-Means Clustering.
* ##### The basic idea behind K-means clustering is to partition an image into k clusters based on pixel similarities.
* ##### Each cluster represents a segment in the image, and pixels within the same cluster share similar characteristics.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html)
* * ![image info](https://miro.medium.com/max/699/1*XbIyaCstQ2-k0dSmdDYhog.png)



In [69]:
@error_handler
def kmeans_clustering(image_path):
    img = cv.imread(image_path)
    z = img.reshape((-1, 3))
    # convert to np.float32
    # define criteria, number of clusters(K)and apply kmeans()
    z = np.float32(z)

    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    clusters = [1, 2, 4, 8, 10, 12, 20]
    for i, k in enumerate(clusters):
        ret, label, center = cv.kmeans(z, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
        # Now convert back into uint8, and make original image
        center = np.uint8(center)
        res = center[label.flatten()]
        res2 = res.reshape(img.shape)
        cv.imshow(f'{k}', res2)
        cv.waitKey(0)
    cv.destroyAllWindows()

path = '../Images/home.jpg'
kmeans_clustering(path)

### Section 2, _Background Subtraction_:
* ##### It is used to identify moving objects within a video sequence by isolating them from the background
* ##### The primary goal of background subtraction is to segment the foreground (moving objects) from the stationary background.
* ##### This technique finds applications in various domains such as surveillance, object tracking, human-computer interaction, and video analysis.
* ##### [Click to check the documentation](https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_video/py_bg_subtraction/py_bg_subtraction.html)

In [71]:
@error_handler
def background_subtraction():
    cap = cv.VideoCapture(0)
    fg_bg = cv.createBackgroundSubtractorMOG2()
    # fgbg=cv.bgsegm.createBackgroundSubtractorGMG()
    # fgbg=cv.bgsegm.createBackgroundSubtractorMOG()
    while True:
        ret, frame = cap.read()
        fg_mask = fg_bg.apply(frame)
        cv.imshow('foreground mask', fg_mask)
        cv.imshow('frame', frame)
        k = cv.waitKey(30)
        get_updated_value(k, 0)
    cap.release()
    cv.destroyAllWindows()

background_subtraction()

Terminated by user!


### Section 3, _Motion Detection_:
* ##### It's the process of identifying and tracking movement or changes in a sequence of images or video frames.
* ##### The goal is to detect regions or objects that are in motion, distinguishing them from the static background.
* ##### [Click to check the course](https://www.coursera.org/learn/object-tracking-and-motion-computer-vision)

In [73]:
@error_handler
def motion_detection():
    cap = cv.VideoCapture(0)
    ret, prev = cap.read()
    ret, current = cap.read()
    while ret:
        diff = cv.absdiff(prev, current)
        gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
        blur = cv.GaussianBlur(gray, (5, 5), 0)
        ret, thresh = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
        dilated = cv.dilate(thresh, np.ones((7, 7), np.uint8), iterations=9)
        contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        cv.drawContours(prev, contours, -1, (0, 255, 0), 2)
        cv.imshow("inter", prev)
        if cv.waitKey(40) == 27:
            break
        prev = current
        ret, current = cap.read()
    cv.destroyAllWindows()
    cap.release()

motion_detection()

## _The End_.